# <span style="color:red; font-size:36px;">1：运行下载模型以及仓库（首次需运行）</span>


In [ ]:
import os
!git clone --branch 1.0.1 https://github.com/Stardust-minus/Bert-VITS2.git

%cd Bert-VITS2/

!wget -q https://huggingface.co/datasets/digitalxingtong/Bert-vits2/resolve/main/pytorch_model.bin;
!wget -q https://huggingface.co/datasets/digitalxingtong/Bert-vits2/resolve/main/xingtong25680.pth;
!wget -q https://huggingface.co/datasets/digitalxingtong/Bert-vits2/resolve/main/tf_model.h5;
!wget -q https://huggingface.co/datasets/digitalxingtong/Bert-vits2/resolve/main/flax_model.msgpack;
!wget -q https://huggingface.co/datasets/digitalxingtong/Bert-vits2/resolve/main/config_.json;
!wget -q https://huggingface.co/datasets/digitalxingtong/Bert-vits2/resolve/main/config.json;
!wget -q https://huggingface.co/datasets/digitalxingtong/Bert-vits2/resolve/main/nailv_read.pth
!wget -q https://huggingface.co/datasets/digitalxingtong/Bert-vits2/resolve/main/nailv_read_config.json
!wget -q https://huggingface.co/datasets/digitalxingtong/Bert-vits2/resolve/main/nailv_zatan.pth
!wget -q https://huggingface.co/datasets/digitalxingtong/Bert-vits2/resolve/main/nailv_zatan_config.json
# !wget -q https://huggingface.co/spaces/digitalxingtong/Jiaran-Bert-VITS2/resolve/main/logs/jiaran/jiaran_new.pth
# !wget -q -O jiaran_config.json https://huggingface.co/spaces/digitalxingtong/Jiaran-Bert-VITS2/resolve/main/logs/jiaran/config.json


file_names = ['pytorch_model.bin', 'xingtong25680.pth', 'tf_model.h5', 'flax_model.msgpack', 'config_.json', 'config.json','nailv_read.pth','nailv_zatan.pth']
suc=True
for file_name in file_names:
    if os.path.exists(file_name):
        pass
    else:
        print(f"{file_name} 下载失败")
        suc=False
if suc:
  print("全部模型下载成功")

# <span style="color:red; font-size:36px;">2：安装模型(首次需运行)</span>


In [ ]:
os.makedirs('logs/xt', exist_ok=True)
import shutil
shutil.move('config.json', 'logs/xt')
os.remove('configs/config.json')
shutil.move('config_.json', 'configs')
os.rename('configs/config_.json', 'configs/config.json')
shutil.move('flax_model.msgpack', 'bert/chinese-roberta-wwm-ext-large')
shutil.move('pytorch_model.bin', 'bert/chinese-roberta-wwm-ext-large')
shutil.move('tf_model.h5', 'bert/chinese-roberta-wwm-ext-large')

shutil.move('xingtong25680.pth', 'logs/xt')

os.makedirs('logs/nailv_read', exist_ok=True)
shutil.move('nailv_read.pth', 'logs/nailv_read')
shutil.move('nailv_read_config.json', 'logs/nailv_read')
os.rename('logs/nailv_read/nailv_read_config.json', 'logs/nailv_read/config.json')


os.makedirs('logs/nailv_zatan', exist_ok=True)
shutil.move('nailv_zatan.pth', 'logs/nailv_zatan')
shutil.move('nailv_zatan_config.json', 'logs/nailv_zatan')
os.rename('logs/nailv_zatan/nailv_zatan_config.json', 'logs/nailv_zatan/config.json')

!pip install -r requirements.txt > /dev/null


import sys, os

device = (
    "cuda:0"

)
import utils
import logging

logging.getLogger("numba").setLevel(logging.WARNING)
logging.getLogger("markdown_it").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

logging.basicConfig(level=logging.INFO, format="| %(name)s | %(levelname)s | %(message)s")

logger = logging.getLogger(__name__)
config_file = "./configs/config.json"
hps = utils.get_hparams_from_file(config_file)

import torch
import argparse
import commons

from models import SynthesizerTrn
from text.symbols import symbols
from text import cleaned_text_to_sequence, get_bert
from text.cleaner import clean_text
import gradio as gr
import webbrowser

net_g = None



def get_text(text, language_str, hps):
    norm_text, phone, tone, word2ph = clean_text(text, language_str)
    phone, tone, language = cleaned_text_to_sequence(phone, tone, language_str)

    if hps.data.add_blank:
        phone = commons.intersperse(phone, 0)
        tone = commons.intersperse(tone, 0)
        language = commons.intersperse(language, 0)
        for i in range(len(word2ph)):
            word2ph[i] = word2ph[i] * 2
        word2ph[0] += 1
    bert = get_bert(norm_text, word2ph, language_str)
    del word2ph

    assert bert.shape[-1] == len(phone)

    phone = torch.LongTensor(phone)
    tone = torch.LongTensor(tone)
    language = torch.LongTensor(language)

    return bert, phone, tone, language

def infer(text, sdp_ratio, noise_scale, noise_scale_w, length_scale, sid):
    global net_g
    bert, phones, tones, lang_ids = get_text(text, "ZH", hps)
    with torch.no_grad():
        x_tst=phones.to(device).unsqueeze(0)
        tones=tones.to(device).unsqueeze(0)
        lang_ids=lang_ids.to(device).unsqueeze(0)
        bert = bert.to(device).unsqueeze(0)
        x_tst_lengths = torch.LongTensor([phones.size(0)]).to(device)
        del phones
        speakers = torch.LongTensor([hps.data.spk2id[sid]]).to(device)
        audio = net_g.infer(x_tst, x_tst_lengths, speakers, tones, lang_ids, bert, sdp_ratio=sdp_ratio
                           , noise_scale=noise_scale, noise_scale_w=noise_scale_w, length_scale=length_scale)[0][0,0].data.cpu().float().numpy()
        del x_tst, tones, lang_ids, bert, x_tst_lengths, speakers
        return audio

def tts_fn(text, speaker, sdp_ratio, noise_scale, noise_scale_w, length_scale):
    with torch.no_grad():
        audio = infer(text, sdp_ratio=sdp_ratio, noise_scale=noise_scale, noise_scale_w=noise_scale_w, length_scale=length_scale, sid=speaker)
    return "Success", (hps.data.sampling_rate, audio)


net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).to(device)
_ = net_g.eval()




# <span style="color:red; font-size:36px;">2.1：安装模型(首次无需运行，第二次打开直接从这里开始运行)</span>


In [ ]:
%cd Bert-VITS2/

!pip install -r requirements.txt > /dev/null

import sys, os

device = (
    "cuda:0"

)
import utils
import logging

logging.getLogger("numba").setLevel(logging.WARNING)
logging.getLogger("markdown_it").setLevel(logging.WARNING)
logging.getLogger("urllib3").setLevel(logging.WARNING)
logging.getLogger("matplotlib").setLevel(logging.WARNING)

logging.basicConfig(level=logging.INFO, format="| %(name)s | %(levelname)s | %(message)s")

logger = logging.getLogger(__name__)
config_file = "./configs/config.json"
hps = utils.get_hparams_from_file(config_file)

import torch
import argparse
import commons

from models import SynthesizerTrn
from text.symbols import symbols
from text import cleaned_text_to_sequence, get_bert
from text.cleaner import clean_text
import gradio as gr
import webbrowser

net_g = None



def get_text(text, language_str, hps):
    norm_text, phone, tone, word2ph = clean_text(text, language_str)
    phone, tone, language = cleaned_text_to_sequence(phone, tone, language_str)

    if hps.data.add_blank:
        phone = commons.intersperse(phone, 0)
        tone = commons.intersperse(tone, 0)
        language = commons.intersperse(language, 0)
        for i in range(len(word2ph)):
            word2ph[i] = word2ph[i] * 2
        word2ph[0] += 1
    bert = get_bert(norm_text, word2ph, language_str)
    del word2ph

    assert bert.shape[-1] == len(phone)

    phone = torch.LongTensor(phone)
    tone = torch.LongTensor(tone)
    language = torch.LongTensor(language)

    return bert, phone, tone, language

def infer(text, sdp_ratio, noise_scale, noise_scale_w, length_scale, sid):
    global net_g
    bert, phones, tones, lang_ids = get_text(text, "ZH", hps)
    with torch.no_grad():
        x_tst=phones.to(device).unsqueeze(0)
        tones=tones.to(device).unsqueeze(0)
        lang_ids=lang_ids.to(device).unsqueeze(0)
        bert = bert.to(device).unsqueeze(0)
        x_tst_lengths = torch.LongTensor([phones.size(0)]).to(device)
        del phones
        speakers = torch.LongTensor([hps.data.spk2id[sid]]).to(device)
        audio = net_g.infer(x_tst, x_tst_lengths, speakers, tones, lang_ids, bert, sdp_ratio=sdp_ratio
                           , noise_scale=noise_scale, noise_scale_w=noise_scale_w, length_scale=length_scale)[0][0,0].data.cpu().float().numpy()
        del x_tst, tones, lang_ids, bert, x_tst_lengths, speakers
        return audio

def tts_fn(text, speaker, sdp_ratio, noise_scale, noise_scale_w, length_scale):
    with torch.no_grad():
        audio = infer(text, sdp_ratio=sdp_ratio, noise_scale=noise_scale, noise_scale_w=noise_scale_w, length_scale=length_scale, sid=speaker)
    return "Success", (hps.data.sampling_rate, audio)


net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model).to(device)
_ = net_g.eval()




# <span style="color:red; font-size:36px;">3：选择模型</span>

In [ ]:
import ipywidgets as widgets
from IPython.display import display

names = ['星瞳', '奶绿阅读', '奶绿杂谈']

dropdown = widgets.Dropdown(
    options=names,  # 设置选项
    value='星瞳',
    description='请选择模型:',  # 设置描述
    disabled=False  # 是否禁用下拉选择框
)
def on_dropdown_change(change):
    selected_name = change['new']
    print(f'You selected: {selected_name}')

dropdown.observe(on_dropdown_change, names='value')
display(dropdown)

# <span style="color:red; font-size:36px;">3.1：更改文本</span>


In [ ]:
import ipywidgets as widgets
from IPython.display import display

speakText = widgets.Textarea(
    value='哪位大师给算一下啊，就是为啥晚上出去看不到太阳啊？？？好苦恼哦。。。不明白！！！',  # 默认值
    description='请输入文字:',  # 设置描述
    disabled=False  # 是否禁用输入框
)

def on_text_change(change):
    entered_text = change['new']
    print(f'You entered: {entered_text}')

speakText.observe(on_text_change, names='value')
display(speakText)

# <span style="color:red; font-size:36px;">3.2：生成音声</span>


In [ ]:
# 显示下拉选择框
if dropdown.value=='星瞳':
  _ = utils.load_checkpoint("./logs/xt/xingtong25680.pth", net_g, None, skip_optimizer=True)
elif dropdown.value=='奶绿阅读':
  _ = utils.load_checkpoint("./logs/nailv_read/nailv_read.pth", net_g, None, skip_optimizer=True)
elif dropdown.value=='奶绿杂谈':
  _ = utils.load_checkpoint("./logs/nailv_zatan/nailv_zatan.pth", net_g, None, skip_optimizer=True)
# else:
#   _ = utils.load_checkpoint("./logs/jr/nailv_zatan.pth", net_g, None, skip_optimizer=True)


speaker_ids = hps.data.spk2id
speakers = list(speaker_ids.keys())
import numpy as np
from scipy.io.wavfile import write


sdp_ratio=0.2 #语调变化
noise_scale=1 #感情变化
noise_scale_w=0.8 #音节发音长度变化
length_scale=1  #更改语速

audio_output = tts_fn(speakText.value, "ign", sdp_ratio, noise_scale, noise_scale_w, length_scale)
_, audio_data1 = audio_output
sample_rate, audio_data = audio_data1
output_filename = "tmp.wav"
write(output_filename, sample_rate, audio_data.astype(np.float32))
from IPython.display import Audio
Audio(output_filename)

# <span style="color:red; font-size:36px;">4：上传长文本</span>


In [ ]:
import ipywidgets as widgets
from IPython.display import display

text_input = widgets.Text(
    value='text',  # 默认值
    description='输入上传的文件路径:',  # 设置描述
    disabled=False  # 是否禁用输入框
)

def on_text_change(change):
    entered_text = change['new']
    print(f'You entered: {entered_text}')

text_input.observe(on_text_change, names='value')
display(text_input)

# <span style="color:red; font-size:36px;">4.1：生成长文本</span>

In [ ]:
if dropdown.value=='星瞳':
  _ = utils.load_checkpoint("./logs/xt/xingtong25680.pth", net_g, None, skip_optimizer=True)
elif dropdown.value=='奶绿阅读':
  _ = utils.load_checkpoint("./logs/nailv_read/nailv_read.pth", net_g, None, skip_optimizer=True)
else:
  _ = utils.load_checkpoint("./logs/nailv_zatan/nailv_zatan.pth", net_g, None, skip_optimizer=True)




speaker_ids = hps.data.spk2id
speakers = list(speaker_ids.keys())
import numpy as np
from scipy.io.wavfile import write


sdp_ratio=0.3
noise_scale=0.4
noise_scale_w=0.9
length_scale=0.99  #更改语速


import re
import os

!pip install -q pydub
folder_name = "out"

if not os.path.exists(folder_name):
    os.makedirs(folder_name)

import numpy as np
from scipy.io.wavfile import write
with open("/kaggle/input/"+text_input.value+".txt", "r", encoding="utf-8") as file:
    content = file.read()

content = re.sub(r'\s+', ' ', content).strip()
content = content.replace('“', '')
content = content.replace('”', '')
content = content.replace('——', '，')
content = content.replace('......', '，')
paragraphs = []

pattern = r'[^，。！？；：、]*[，。！？；：、][^，。！？；：、]*[，。！？；：、][^，。！？；：、]*[，。！？；：、][^，。！？；：、]*[，。！？；：、][^，。！？；：、]*[，。！？；：、][^，。！？；：、]*[，。！？；：、]'
matches = re.finditer(pattern, content)
start = 0
for match in matches:
    paragraph = match.group(0).strip()
    paragraphs.append(paragraph)
    start = match.end()

if start < len(content):
    paragraph = content[start:].strip()
    paragraphs.append(paragraph)

cnt=0
for paragraph in paragraphs:
    text=paragraph
    audio_output = tts_fn(text, "ign", sdp_ratio, noise_scale, noise_scale_w, length_scale)
    _, audio_data1 = audio_output
    sample_rate, audio_data = audio_data1
    output_filename = f"out/{cnt}.wav"
    write(output_filename, sample_rate, audio_data.astype(np.float32))
    cnt+=1

code = """
from pydub import AudioSegment


input_folder = "out"
output_file = "长文本.wav"

output_audio = AudioSegment.empty()

for i in range(10000):
    input_file = f"{input_folder}/{i}.wav"
    pause_duration=50
    try:
        audio_segment = AudioSegment.from_wav(input_file)
        if len(output_audio) > 0:
            pause = AudioSegment.silent(duration=pause_duration)
            output_audio += pause
        output_audio += audio_segment
    except FileNotFoundError:

        continue

output_audio.export(output_file, format="wav")

"""
with open("merge.py", "w") as file:
    file.write(code)
%run merge.py



# <span style="color:red; font-size:36px;">4.2：运行下载长文本</span>


In [ ]:
Audio('长文本.wav')